In [2]:
import xml.etree.ElementTree as ET
from datetime import datetime
from googlesearch import search
import time
import requests
import json
import re
from bs4 import BeautifulSoup
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter

c:\users\ramon\appdata\local\programs\python\python39\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.8) or chardet (5.1.0)/charset_normalizer (2.0.11) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "


In [86]:
from rdflib import Graph, URIRef

# Create an instance of the Graph class
g = Graph()

# Read the TTL file into the graph
g.parse('rwc_zbo_compleet_1.ttl', format='turtle')

# Get the sorted list of triples
sorted_triples = sorted(g, key=lambda triple: triple)


ZB = dict()
# Iterate over the sorted triples and print them
for subj, pred, obj in sorted_triples:
    if pred == URIRef('http://www.w3.org/2000/01/rdf-schema#label'):
        code = subj.split('/')[-1]
        ZB[code] = str(obj)
    

In [87]:
with open('alle_organisaties.json', 'r') as f:
    json_data = json.load(f)

columns = ['dc_identifier', 'dc_title', 'dc_publisher', 'dc_creator', 'foi_retrievedDate', 'dc_date_year', 'dc_publisher_name', 'foi_website', 'foaf_mbox', 'foaf_phone', 'foi_bezoekadres', 'foi_postadres', 'foi_fax', 'foi_startDate', 'foi_endDate', 'foi_lastUpdate', 'foaf_type', 'foi_files']

    
# Extract the "foi_dossiers" nested dictionary
foi_dossiers = json_data['infobox']['foi_dossiers']

data = {}

# Iterate over the keys and values of the "foi_dossiers" dictionary
for key, value in foi_dossiers.items():
    data[key] = value.values()
    
# Convert the modified data to a DataFrame
df = pd.DataFrame(pd.DataFrame.from_dict(data, orient='index', columns=columns))
display(df)

,dc_identifier,dc_title,dc_publisher,dc_creator,foi_retrievedDate,dc_date_year,dc_publisher_name,foi_website,foaf_mbox,foaf_phone,foi_bezoekadres,foi_postadres,foi_fax,foi_startDate,foi_endDate,foi_lastUpdate,foaf_type,foi_files
0,nl.oorg10122,Agentschap - Rijksdienst voor Ondernemend Nede...,oorg10122,Ramon Duursma,2023-06-08,2023,Rijksdienst voor Ondernemend Nederland,https://www.rvo.nl/,,(088) 042 42 42,Prinses Beatrixlaan 2 2595 AL Den Haag,93144 2509 AC Den Haag,,2014-01-01,,2023-01-19,Agentschap,[]
1,nl.,Regionaal samenwerkingsorgaan - HALte werk,,Ramon Duursma,2023-06-08,2023,HALte werk,https://www.haltewerk.nl,,14 072,Mallegatsplein 10 1815 AG Alkmaar,8002 1802 KA Alkmaar,,2015-01-01,,2021-08-30,Regionaal samenwerkingsorgaan,[]
2,nl.oorg10196,Adviescollege - Commissie voor de milieueffect...,oorg10196,Ramon Duursma,2023-06-08,2023,Commissie voor de milieueffectrapportage,https://www.commissiemer.nl/,,,,,,1994-01-01,,2022-04-05,Adviescollege,[]
3,nl.,Regionaal samenwerkingsorgaan - Servicecentrum...,,Ramon Duursma,2023-06-08,2023,Servicecentrum MER,https://www.echt-susteren.nl/servicecentrum-mer,sandra.smeets@servicecentrum-mer.nl,(0475) 25 57 58,Nieuwe Markt 55 6101 CV Echt,450 6100 AL Echt,,2012-10-15,,2022-05-31,Regionaal samenwerkingsorgaan,[]
4,nl.,Regionaal samenwerkingsorgaan - Westrom,,Ramon Duursma,2023-06-08,2023,Westrom,https://www.westrom.nl/,westrom@westrom.nl,(0475) 38 98 98,De Hanze 1 6041 PA Roermond,1117 6040 KC Roermond,,2001-01-01,,2022-10-18,Regionaal samenwerkingsorgaan,[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4004,nl.,Organisatieonderdeel - Vereniging van Limburgs...,,Ramon Duursma,2023-06-08,2023,Vereniging van Limburgse gemeenten,,,,,30435 2500 GK Den Haag,,,,2021-02-27,Organisatieonderdeel,[]
4005,nl.,Organisatieonderdeel - Vereniging van Groninge...,,Ramon Duursma,2023-06-08,2023,Vereniging van Groninger Gemeenten,,,(0599) 63 16 31,Hendrik Westerstraat 24 9791 CT Ten Boer,9 9790 AA Ten Boer,,,,2021-04-16,Organisatieonderdeel,[]
4006,nl.,Organisatieonderdeel - Vereniging van Noord-Ho...,,Ramon Duursma,2023-06-08,2023,Vereniging van Noord-Hollandse Gemeenten,,,,,,,,,2021-04-16,Organisatieonderdeel,[]
4007,nl.,Organisatieonderdeel - Vereniging van Zeeuwse ...,,Ramon Duursma,2023-06-08,2023,Vereniging van Zeeuwse Gemeenten,,,,,6000 4330 LA Middelburg,,,,2021-02-27,Organisatieonderdeel,[]


In [89]:
Counter(df['foaf_type'].to_list())

Counter({'Agentschap': 30,
         'Regionaal samenwerkingsorgaan': 421,
         'Adviescollege': 111,
         'Gemeente': 419,
         'Organisatie met overheidsbemoeienis': 1376,
         'Grensoverschrijdend regionaal samenwerkingsorgaan': 11,
         'Rechtspraak': 25,
         'Zelfstandig bestuursorgaan': 156,
         'Interdepartementale commissie': 7,
         'Koepelorganisatie': 4,
         'Organisatieonderdeel': 1382,
         'Openbaar lichaam voor beroep en bedrijf': 6,
         'Hoog College van Staat': 6,
         'Ministerie': 12,
         'Kabinet van de Koning': 1,
         'Provincie': 12,
         'Waterschap': 25,
         'Caribisch openbaar lichaam': 3,
         'Politie en brandweer': 2})

In [90]:
print(len(ZB.items()))

zonder_zb = []

for c, n in ZB.items():

    result = df[df['dc_publisher_name'] == n]
    
    if result.empty:
        print('')
        print(c,n)
        print('')
        
    else:
        if result.foaf_type[0] == 'Adviescollege':
            print(c,n)
        if result.dc_publisher[0] == '':
            zonder_zb.append(result.dc_publisher_name[0])
            
print(f"{len(zonder_zb)}: organisaties zonder zb in de XML maar wel in de tll")

157
zb000174 Kiesraad

zb000185 Instituut Fysieke Veiligheid

zb000215 Autoriteit Persoonsgegevens
38: organisaties zonder zb in de XML maar wel in de tll
